In [1]:
import numpy as np
import torch
from tsGaussian.torch_tsgaussian import TangentSpaceGaussian
from stable_baselines_utils import TangentSpaceGaussian as TSG
# from pytorch3d.transforms.so3 import (
#     so3_exp_map,
#     so3_relative_angle,
# )

/home/fantasticoven/.local/lib/python3.8/site-packages/tqdm/auto.py:22: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [2]:
tg = TangentSpaceGaussian(None)

# Test liegroup torch

In [3]:
from liegroups.torch import SO3

In [4]:
C = SO3.exp(torch.Tensor([[1,2,3],
                         [0,0,0]]))
C

<liegroups.torch.so3.SO3Matrix>
| tensor([[[-0.6949,  0.7135,  0.0893],
|          [-0.1920, -0.3038,  0.9332],
|          [ 0.6930,  0.6313,  0.3481]],
| 
|         [[ 1.0000,  0.0000,  0.0000],
|          [ 0.0000,  1.0000,  0.0000],
|          [ 0.0000,  0.0000,  1.0000]]])

# Test torch_tsgaussian sample

In [5]:
R_mu = torch.eye(3).reshape((1,3,3))
sigma = torch.ones(3).reshape((1,3))

In [6]:
R_quat, R_x = tg.rsample(R_mu, sigma)

-1


RuntimeError: Device index must not be negative

In [ ]:
torch.bmm(torch.transpose(R_x, 1, 2), R_x)

# Test torch_tsgaussian normal_term

In [ ]:
sigma = torch.ones(3).reshape((1,3))
sigma

In [ ]:
tg.normal_term(sigma)

# Test torch_tsgaussian log_map

In [ ]:
R_1 = torch.eye(3).reshape((1, 3, 3))
R_2 = torch.eye(3).reshape((1, 3, 3))

In [ ]:
tg.log_map(R_1, R_2)

# Test torch_tsgaussian log_probs

In [ ]:
R_x = torch.eye(3).reshape((1,3,3))
R_mu = torch.zeros(3,3).reshape((1,3,3))
R_x = R_x.repeat(5, 1, 1)
R_mu = R_mu.repeat(5, 1, 1)
sigma = torch.ones(3).reshape((1,3))

In [ ]:
# tg.log_probs(R_x, R_mu, sigma)

In [ ]:
np.e ** (-2.7568)

all codes run for torch_tsgaussian now, need to check it's correctness and make it into batch version.

# Test TangentSpaceGaussian actions_from_params

In [ ]:
tsg = TSG(None)

In [ ]:
print(tsg.distribution)

In [ ]:
tsg

In [ ]:
tsg.actions_from_params(torch.eye(3).reshape((1,3,3)), torch.ones(3).reshape((1,3)))

# Test TangentSpaceGaussian log_prob_from_params

In [ ]:
torch.eye(3).repeat(2,1,1).size()

In [ ]:
torch.ones(3).repeat(2,1).size()

In [ ]:
# tsg.log_prob_from_params(torch.eye(3).repeat(2,1,1), torch.ones(3))

In [ ]:
x = torch.randn(2, 4, 4)
y = torch.linalg.inv(x)
y

Again, codes can run, but need to check correctness.

# Try to run training

In [7]:
import torch
from absl import app, flags
from stable_baselines3 import SAC, PPO
from envs.wahba import Wahba
from stable_baselines_utils import CustomSACPolicy, \
    CustomCNN

In [8]:
def main(argv):
    env = Wahba()
    device = torch.device('cpu')
    policy_kwargs = dict(
        features_extractor_class = CustomCNN,
        features_extractor_kwargs = dict(features_dim = 256))
    policy_kwargs['n_critics'] = 1
    policy_kwargs['share_features_extractor'] = False
    policy = CustomSACPolicy
    model = SAC(policy, env, verbose = 1, ent_coef = 'auto_0.1',
                policy_kwargs = policy_kwargs, device = device)
    model.learn(total_timesteps = 500, eval_freq = 100, n_eval_episodes = 100)
    

In [9]:
np.e ** (-8)

0.00033546262790251196

In [12]:
from torch import autograd
with autograd.detect_anomaly():
    env = Wahba()
    device = torch.device(
        "cuda:0") if torch.cuda.is_available() else torch.device("cpu")
    policy_kwargs = dict(
        features_extractor_class = CustomCNN,
        features_extractor_kwargs = dict(features_dim = 256))
    policy_kwargs['n_critics'] = 1
    policy_kwargs['share_features_extractor'] = False
    policy = CustomSACPolicy
    model = SAC(policy, env, verbose = 1, ent_coef = 'auto_0.1',
                policy_kwargs = policy_kwargs, device = device)
    model.learn(total_timesteps = 50000, eval_freq = 100, n_eval_episodes = 100)

Using cuda:0 device
Wrapping the env with a `Monitor` wrapper
Wrapping the env in a DummyVecEnv.
---------------------------------
| rollout/           |          |
|    ep_len_mean     | 1        |
|    ep_rew_mean     | -5.37    |
| time/              |          |
|    episodes        | 4        |
|    fps             | 823      |
|    time_elapsed    | 0        |
|    total_timesteps | 4        |
---------------------------------
---------------------------------
| rollout/           |          |
|    ep_len_mean     | 1        |
|    ep_rew_mean     | -4.73    |
| time/              |          |
|    episodes        | 8        |
|    fps             | 877      |
|    time_elapsed    | 0        |
|    total_timesteps | 8        |
---------------------------------
---------------------------------
| rollout/           |          |
|    ep_len_mean     | 1        |
|    ep_rew_mean     | -5.3     |
| time/              |          |
|    episodes        | 12       |
|    fps           

---------------------------------
| rollout/           |          |
|    ep_len_mean     | 1        |
|    ep_rew_mean     | -5.75    |
| time/              |          |
|    episodes        | 100      |
|    fps             | 755      |
|    time_elapsed    | 0        |
|    total_timesteps | 100      |
---------------------------------


/tmp/ipykernel_6623/1732186999.py:2: UserWarning: Anomaly Detection has been enabled. This mode will increase the runtime and should only be enabled for debugging.
  with autograd.detect_anomaly():


0
0
0
0
0
0
0
0
0
0
---------------------------------
| rollout/           |          |
|    ep_len_mean     | 1        |
|    ep_rew_mean     | -5.77    |
| time/              |          |
|    episodes        | 104      |
|    fps             | 162      |
|    time_elapsed    | 0        |
|    total_timesteps | 104      |
| train/             |          |
|    actor_loss      | 0.311    |
|    critic_loss     | 17.6     |
|    ent_coef        | 0.1      |
|    ent_coef_loss   | -3.65    |
|    learning_rate   | 0.0003   |
|    n_updates       | 3        |
---------------------------------
0
0
0
0
0
0
0
0
0
0
0
0
---------------------------------
| rollout/           |          |
|    ep_len_mean     | 1        |
|    ep_rew_mean     | -5.81    |
| time/              |          |
|    episodes        | 108      |
|    fps             | 82       |
|    time_elapsed    | 1        |
|    total_timesteps | 108      |
| train/             |          |
|    actor_loss      | 0.363    |
|   

KeyboardInterrupt: 

In [ ]:
model

In [ ]:
random_action = env.action_space.sample()
random_action

In [ ]:
obs = env.step(random_action)[0]
obs.shape

In [ ]:
act = model.predict(obs)

In [ ]:
act

In [ ]:
%load_ext tensorboard
import tensorflow as tf
import numpy as np
import datetime
import matplotlib.pyplot as plt
from tensorboard.backend.event_processing import event_accumulator

In [ ]:
%tensorboard --logdir ./sac

# Experiments for batch operations

In [ ]:
sigma = torch.ones(1, 3)
omiga = torch.normal(torch.zeros(1, 3), sigma)
omiga

In [ ]:
def transfer(omiga):
    omiga_0, omiga_1, omiga_2 = omiga[0], omiga[1], omiga[2]
    omiga_hat = torch.tensor([[0, -omiga_2, omiga_1],
                                [omiga_2, 0, -omiga_0],
                                [-omiga_1, omiga_0, 0]])
    return omiga_hat

In [ ]:
from functorch import vmap
batch_transfer = vmap(transfer)
batch_transfer(omiga)

In [ ]:
from liegroups.torch import SO3
C = SO3.exp(torch.Tensor([[1,2,3],
                          [0,0,0]]))
print(torch.Tensor([[1,2,3],
                          [0,0,0]]).size())
SO3.log(C)

In [ ]:
np.log(1)

# Question to ask: the original wahba problem action is (4,), in our case actions are (3,3).